# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
get_files('data/song_data')

['/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAMO128F1481E7F.json',
 '/home/workspace/data/song_data/A/A/A/TRAAABD128F429CF47.json',
 '/home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVO128F93133D4.json',
 '/home/workspace/data/song_data/A/A/A/TRAAADZ128F9348C2E.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAPK128E0786D96.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAEF128F4273421.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json',
 '/home/workspace/data/song_data/A/A/B/TRAABYW128F4244559.json',
 '/home/workspace/data/song_data/A/A/B/TRAABRB128F9306DD5.json',
 '/home/workspace/data/song_data/A/A/B/TRAABLR128F423B7E3.json',
 '/home/workspace/data/song_data/A/A/B/TRAABDL12903CAABBA.json',
 '/home/workspace/data/so

In [5]:
song_files = ('/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json')

In [6]:
filepath = ('data/song_data')

In [7]:
df = pd.read_json(song_files, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD0S291187B9B7BF5,NaN,Ohio,NaN,Rated R,114.78159,1,SOMJBYD12A6D4F8557,Keepin It Real (Skit),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [10]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0]
song_data = song_data.tolist()
type(song_data)

list

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [11]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0]
artist_data = artist_data.tolist()
artist_data

['ARD0S291187B9B7BF5', 'Rated R', 'Ohio', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
get_files('data/log_data')

['/home/workspace/data/log_data/2018/11/2018-11-29-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-03-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-01-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-05-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-02-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-24-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-13-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-26-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-06-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-14-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-23-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-09-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-10-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-17-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-20-events.json',
 '/home/workspace/data/lo

In [15]:
log_files = '/home/workspace/data/log_data/2018/11/2018-11-29-events.json'

In [16]:
filepath = 'data/log_data'

In [31]:
df = pd.read_json(log_files, lines=True)
df.tail()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
361,None,Logged In,Tegan,F,37,Levine,NaN,paid,"Portland-South Portland, ME",GET,Home,1.540794e+12,1065,None,200,1543532195796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
362,Pearl Jam,Logged In,Tegan,F,38,Levine,283.89832,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Rearviewmirror,200,1543532242796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
363,The 2 Live Crew,Logged In,Tegan,F,39,Levine,188.31628,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Me So Horny,200,1543532525796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
364,311,Logged In,Tegan,F,40,Levine,113.26649,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Loco,200,1543532713796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
365,Light Of The World,Logged In,Kynnedi,F,0,Sanchez,207.07220,free,"Cedar Rapids, IA",PUT,NextSong,1.541079e+12,1029,"Midnight Groovin' (7"" Version) (2006 Digital R...",200,1543533515796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5...",89


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [32]:
df = df[df['page'] == 'NextSong']
df.tail()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
360,Taylor Swift,Logged In,Tegan,F,36,Levine,230.47791,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,You Belong With Me,200,1543532012796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
362,Pearl Jam,Logged In,Tegan,F,38,Levine,283.89832,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Rearviewmirror,200,1543532242796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
363,The 2 Live Crew,Logged In,Tegan,F,39,Levine,188.31628,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Me So Horny,200,1543532525796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
364,311,Logged In,Tegan,F,40,Levine,113.26649,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,1065,Loco,200,1543532713796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
365,Light Of The World,Logged In,Kynnedi,F,0,Sanchez,207.07220,free,"Cedar Rapids, IA",PUT,NextSong,1.541079e+12,1029,"Midnight Groovin' (7"" Version) (2006 Digital R...",200,1543533515796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5...",89


In [34]:
df['ts'] = pd.to_datetime(df['ts'], unit='ms').dt.ceil(freq='s')
#df['ts'] = df.ts.dt.ceil(freq='s') 
print(df['ts'].tail())
print(f"\nLenght of DF - %s rows" %len(df))

360   2018-11-29 22:53:33
362   2018-11-29 22:57:23
363   2018-11-29 23:02:06
364   2018-11-29 23:05:14
365   2018-11-29 23:18:36
Name: ts, dtype: datetime64[ns]

Lenght of DF - 319 rows


In [18]:
time_df = pd.DataFrame(columns=['start_time','hour', 'day', 'week', 'month', 'year', 'weekday'])

series_list = [df['ts'].dt.strftime("%H:%M:%S"),
               df['ts'].dt.hour, 
               df['ts'].dt.day, 
               df['ts'].dt.week, 
               df['ts'].dt.month, 
               df['ts'].dt.year, 
               df['ts'].dt.day_name()]

i = 0

for serie in series_list:
    col = time_df.columns[i]
    time_df[col] = [var for var in serie]
    i += 1

time_df.tail()

,start_time,hour,day,week,month,year,weekday
314,22:53:32,22,29,48,11,2018,Thursday
315,22:57:22,22,29,48,11,2018,Thursday
316,23:02:05,23,29,48,11,2018,Thursday
317,23:05:13,23,29,48,11,2018,Thursday
318,23:18:35,23,29,48,11,2018,Thursday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [19]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [20]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']].drop_duplicates()
user_df['userId'] = user_df['userId'].astype(int)

user_df.sort_values(by='userId', inplace=True)

print(user_df.head())
print(f"\nNumber of unique Users - %s" %len(user_df))

     userId firstName lastName gender level
42        4    Alivia  Terrell      F  free
337       9     Wyatt    Scott      M  free
56       10    Sylvie     Cruz      F  free
328      12    Austin  Rosales      M  free
46       14  Theodore   Harris      M  free

Number of unique Users - 27


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [21]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [25]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = [row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.